# Import Setup AC
This example shows how to import SIwave, HFSS setups for AC analysis. In this example, we are going to

- Download an example board
- Create a configuration file
  - add setups
- Import the configuration file

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import toml
from pathlib import Path
import tempfile

from ansys.aedt.core.examples.downloads import download_file
from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

Download the example PCB data.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", local_path=temp_folder.name)

## Load example layout.

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:29:48.281739


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


...

PyEDB INFO: Database ANSYS-HSD_V1.aedb Opened in 2025.2


PyEDB INFO: Cell main Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 9.0228 seconds.


PyEDB INFO: EDB initialization completed in 9.0863 seconds.


## Create an empty dictionary to host all configurations.

In [5]:
cfg = dict()

## Create an SIwave SYZ setup

Keywords

- **name**. Name of the setup.
- **type**. Type of the analysis setup. Supported types are 'siwave_ac', 'siwave_dc', 'hfss'.
- **pi_slider_position**. PI slider position. Supported values are from '0', '1', '2'. 0:speed, 1:balanced,
2:accuracy.
- **freq_sweep**. List of frequency sweeps.
  - **name**. Name of the sweep.
  - **type**. Type of the sweep. Supported types are 'interpolation', 'discrete', 'broadband'.
  - **frequencies**. Frequency distribution.
    - **distribution**. Supported distributions are 'linear_count', 'linear_scale', 'log_scale'.
    - **start**. Start frequency. Example, 1e6, "1MHz".
    - **stop**. Stop frequency. Example, 1e9, "1GHz".
    - **increment**.

In [6]:
siwave_setup = {
    "name": "siwave_1",
    "type": "siwave_ac",
    "pi_slider_position": 1,
    "freq_sweep": [
        {
            "name": "Sweep1",
            "type": "interpolation",
            "frequencies": [{"distribution": "log_scale", "start": 1e6, "stop": 1e9, "increment": 20}],
        }
    ],
}

## Create a HFSS setup

Keywords

- **name**. Name of the setup.
- **type**. Type of the analysis setup. Supported types are 'siwave_ac', 'siwave_dc', 'hfss'.
- **f_adapt**. Adaptive frequency.
- **max_num_passes**. Maximum number of passes.
- **max_mag_delta_s**. Convergence criteria delta S.
- **mesh_operations**. Mesh operations.
    - **name**. Name of the mesh operation.
    - **type**. Type of the mesh operation. The supported types are 'base', 'length', 'skin_depth'.
    - **max_length**. Maximum length of elements.
    - **restrict_length**. Whether to restrict length of elements.
    - **refine_inside**. Whether to turn on refine inside objects.
    - **nets_layers_list**. {'layer_name':['net_name_1', 'net_name_2']}
- **freq_sweep**. List of frequency sweeps.
  - **name**. Name of the sweep.
  - **type**. Type of the sweep. Supported types are 'interpolation', 'discrete', 'broadband'.
  - **frequencies**. Frequency distribution.
    - **distribution**. Supported distributions are 'linear_count', 'linear_scale', 'log_scale'.
    - **start**. Start frequency. Example, 1e6, "1MHz".
    - **stop**. Stop frequency. Example, 1e9, "1GHz".
    - **increment**.

In [7]:
hfss_setup = {
    "name": "hfss_1",
    "type": "hfss",
    "f_adapt": "5GHz",
    "max_num_passes": 10,
    "max_mag_delta_s": 0.02,
    "mesh_operations": [
        {
            "name": "mop_1",
            "type": "length",
            "max_length": "3mm",
            "restrict_length": True,
            "refine_inside": False,
            "nets_layers_list": {"GND": ["1_Top", "16_Bottom"]},
        }
    ],
    "freq_sweep": [
        {
            "name": "Sweep1",
            "type": "interpolation",
            "frequencies": [{"distribution": "log_scale", "start": 1e6, "stop": 1e9, "increment": 20}],
        }
    ],
}

## Add setups in configuration

In [8]:
cfg["setups"] = [siwave_setup, hfss_setup]

## Write configuration into as json file

In [9]:
file_json = Path(temp_folder.name) / "edb_configuration.json"
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Equivalent toml file looks like below 

In [10]:
toml_string = toml.dumps(cfg)
print(toml_string)

[[setups]]
name = "siwave_1"
type = "siwave_ac"
pi_slider_position = 1
[[setups.freq_sweep]]
name = "Sweep1"
type = "interpolation"
[[setups.freq_sweep.frequencies]]
distribution = "log_scale"
start = 1000000.0
stop = 1000000000.0
increment = 20



[[setups]]
name = "hfss_1"
type = "hfss"
f_adapt = "5GHz"
max_num_passes = 10
max_mag_delta_s = 0.02
[[setups.mesh_operations]]
name = "mop_1"
type = "length"
max_length = "3mm"
restrict_length = true
refine_inside = false

[setups.mesh_operations.nets_layers_list]
GND = [ "1_Top", "16_Bottom",]
[[setups.freq_sweep]]
name = "Sweep1"
type = "interpolation"
[[setups.freq_sweep.frequencies]]
distribution = "log_scale"
start = 1000000.0
stop = 1000000000.0
increment = 20






## Import configuration into example layout

In [11]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00


PyEDB INFO: Placing sources finished. Time lapse 0:00:00


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Placing ports finished. Time lapse 0:00:00


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


True

## Review

In [12]:
edbapp.setups

{'hfss_1': <pyedb.dotnet.database.utilities.hfss_simulation_setup.HfssSimulationSetup at 0x2479553e440>,
 'siwave_1': <pyedb.dotnet.database.utilities.siwave_simulation_setup.SiwaveSimulationSetup at 0x2479553d0c0>}

## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace **edbapp.save()** with
**edbapp.save_as("C:/example.aedb")** to keep the example project.

In [13]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.1234 seconds.


PyEDB INFO: Close Edb file completed in 0.0662 seconds.


True